# Decision Tree

## Features of decision tree
* A tree with internal nodes labeled by terms
* Branches are labeled by tests on the weight that the term has (or just presence/absence)
* Leaves are labeled by categories
* Classifier categorizes document by descending tree following tests to leaf
* The label of the leaf node is then assigned to the document
* Most decision trees are binary trees (never disadvantageous; may require extra internal nodes)

## Training Process
* Learn a sequence of tests on features, typically using top-down, greedy search
* At each stage choose unused feature with highest Information Gain
* At leaves, either categorical (yes/no) or continuous decisions

### Find best feature for node
Recursively choosing a best split feature at each node. A good features splits the examples into subsets that are
ideally all positive or all negative. So we use information gain as a measurement to find the best feature.

Entropy is defined at each node:
* $p_i$ be the fraction of examples in class i
* $p_i^f$ be the fraction of elements with feature f that lie in class i
* $p_i^{\neg f}$ be the fraction of elements without feature f that lie in class i
* let $p^f$ and $p^{\neg f}$ be the fraction of nodes with (respectively without) feature f

<img src="./img/info_gain.jpeg" width="550">

At each node, we choose the feature f which maximizes the information gain.

This tends to be produce mixtures of classes at each node that are more and more “pure” as you go down the tree.

If a node has examples all of one class c, we make it a leaf and output “c”. Otherwise, we potentially continue to build.

If a leaf still has a mixed distribution, we output the most popular class at that node.

If the features are numeric, we could discretize into bins:
* Divide all numeric values into k bins.
* Feature is treated as if categorical
* Binning can be based on statistics of the entire dataset
* For instance one might use k-means clustering on values of feature

The best thing about decision tree is thatdDecision trees are easily interpreted by humans – much more easily than methods like Naive Bayes, in fact you can extract rules from decision trees

## Simple implementation of decision tree for text classification

In [1]:
# Define the function to calculate entropy
def get_entropy(c):
    e = 0
    s = sum(c.values())
    for l in c.keys():
        p = c[l] / s
        e -= p*math.log2(p)
    return e

In [8]:
import numpy as np
import math
from collections import Counter, defaultdict

class Node():
    """
    Node class for Decision Tree
    find splits with information gain and recursive add nodes to leaf node
    """
    def __init__(self, X, y, features, depth, max_depth, min_gain):
        self.best_feature = None
        self.available_feature = features
        self.depth = depth+1
        self.pos = None
        self.neg = None
        self.is_leaf = False
        self.label = None
        self.data = y
        self.split(X, y, min_gain, max_depth)
        rest_feature = self.available_feature - set([self.best_feature])
        if rest_feature:
            if not self.is_leaf:
                X_pos, X_neg, y_pos, y_neg = [], [], [], []
                for idx in range(len(X)):
                    if self.best_feature in X[idx]:
                        X_pos.append(X[idx])
                        y_pos.append(y[idx])
                    else:
                        X_neg.append(X[idx])
                        y_neg.append(y[idx])
                self.pos = Node(X_pos, 
                                y_pos,
                                rest_feature,
                                self.depth,
                                max_depth, min_gain)
                self.neg = Node(X_neg, 
                                y_neg,
                                rest_feature,
                                self.depth,
                                max_depth, min_gain)

    def split(self, X, y, min_gain, max_depth):
        class_cnt = Counter(y)
        if self.depth <= max_depth:
            most_gain = float('-inf')
            feature_class_cnt = defaultdict(Counter)
            for item in zip(X, y):
                data, label = item
                for feature in data:
                    feature_class_cnt[feature].update([label])
            for feature in self.available_feature:
                if feature_class_cnt[feature]:
                    # this feature has data
                    pos = feature_class_cnt[feature]
                    neg = class_cnt-feature_class_cnt[feature]
                    if not neg:
                        continue
                    e1 = sum(pos.values())*get_entropy(pos) / sum(class_cnt.values())
                    e2 = sum(neg.values())*get_entropy(neg) / sum(class_cnt.values())
                    gain = -(e1+e2)
                    if gain > most_gain:
                        most_gain = gain
                        self.best_feature = feature
        if self.depth > max_depth or get_entropy(class_cnt) + most_gain < min_gain:
            self.is_leaf = True
            self.label = class_cnt.most_common(1)[0][0]

In [10]:
class DecisionTree():
    """
    Class for Decision Tree
    """
    def __init__(self, max_depth, min_gain):
        self.root = None
        self.max_depth = max_depth
        self.min_gain = min_gain

    def train(self, X, y):
        bow = set.union(*X)
        self.root = Node(X, y, bow, 0, self.max_depth, self.min_gain)

    def get_predict(self, x):
        node = self.root
        while node:
            if node.is_leaf:
                return node.label, node.data
            if node.best_feature in x:
                node = node.pos
            else:
                node = node.neg

    def predict(self, X):
        if not self.root:
            return
        pre = []
        for x in X:
            label, data = self.get_predict(x)
            pre.append(label)
        return pre

In [11]:
def parse_line(line):
    """
    Parse one line in the file
    return: set of words, class
    """
    l = line.strip().split()
    tag = l[0]
    word_set = set([w.split(':')[0] for w in l[1:]])
    return word_set, tag

def parse_file(filename):
    """
    Parse file
    return: X - word set list, y - class list
    """
    X, y = [], []
    with open(filename, 'r') as f:
        for line in f.readlines():
            word_set, target = parse_line(line)
            X.append(word_set)
            y.append(target)
    return X, y

training_data = 'data/train.txt'
test_data = 'data/test.txt'

X_train, y_train = parse_file(training_data)
X_test, y_test = parse_file(test_data)

## Train the model and get some evaluation

In [17]:
dt = DecisionTree(10, 0.01)
dt.train(X_train, y_train)
y_pre = dt.predict(X_test)

In [18]:
correct = 0
for item in zip(y_test, y_pre):
    correct += item[0] == item[1]
print(correct / len(y_test))

0.6133333333333333
